In [3]:
# keep this here pls
!pip install xlrd 
#!pip install country_converter

In [ ]:
import pandas as pd
import numpy as np

#define path to xls
url = 'https://www.sharkattackfile.net/spreadsheets/GSAF5.xls'
df = pd.read_excel(url)
df  # 6992 rows × 23 columns

In [5]:
#df.head()

In [6]:
#df.tail()

In [ ]:
df.shape[0]
df.shape[1]

print("Number of Rows: ", df.shape[0])
print("Number of Columns: ", df.shape[1])

df.columns

# Issues w/ og file –> extra 8 columns on VSC? to preserve NaN
# Date, Year, Type, Country, State, Location, Activity, Name, Sex, Age, Injury, Fatal Y/N, Time, Specie, Source
# check search tool between uppercase and lowercase e.g. Australia / AUSTRALIA

In [ ]:
df.columns.str.replace(" ", "_") # Changing all the spaces in the columns with "_"

In [ ]:
df.describe()

In [ ]:
df.dtypes #Here I am reviewing all the data types per column

In [ ]:
df.nunique() # Reviewing here the unique values per column

In [ ]:
# Here I am trying to identify the number of unique values for each column and determine which columns appear to be categorical. 
unique_counts = df.nunique() # Identify the number of unique values for each column
print("\nUnique value counts per column: ")
print(unique_counts)

In [ ]:
#Checking for null values in each column 
df.isna().any()


In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.duplicated().any()

In [ ]:
#Reviewing how many were provoked and unprovoked 
print(df["Type"].unique())
df["Type"] = df["Type"].replace({
    " Provoked": "Provoked", 
    "Unconfirmed": "Unverified",
    "?" : "Unverified",
    "Questionable": "Unverified"
})

In [ ]:
df_nan = df[df["Type"].isna()]
df_nan

# What is the meaning in here as "Provoked"?? 

In [ ]:
df.describe()

In [ ]:
df.nunique() # Reviewing here the number of unique values per column

In [21]:
#for col in df.columns: 
#    if df[col].dtype == object: 
#        print(col)
#        print(df[col].nunique())
#        print(df[col].unique())
#    else: 
#        print(col)
#        print(df[col].nunique())
#        print(df[col].min(), df[col].max)

In [22]:
df["Sex"] = df["Sex"].replace({" M": "M", "M ": "M", "M x 2": "M", " nan": "Unknown", "lli": "Unknown", "N": "M", ".": "Unknown", " nan":"Unknown"})

In [23]:
df["Sex"] = df["Sex"].fillna("Unknown")

In [ ]:
df.Sex.unique()

In [ ]:
df["Species"] = df["Species "].str.extract(r'([A-Za-z\s-]+)').fillna("Not stated") # Cleaning here the unnecessary details
df

In [ ]:
#df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
df

In [ ]:
#To Reemove Extra Spaces and Fix Captilazation

df["Country"] = df["Country"].str.strip().str.title()
df

In [ ]:
df.Country.unique()

In [ ]:
#df["Country"] = df["Country"].replace({
    
})

In [ ]:
df["Country"] = df["Country"].str.replace(r"[^a-zA-Z\s]", "", regex=True)

In [ ]:
print(df["Country"].unique())

In [ ]:
# Standardize country names using country_converter
df["Country"] = df["Country"].apply(lambda x: coco.convert(x, to="name_short"))
df